In [1]:
try:
   import syft as sy
   print("Syft is imported")
except:
   print("Syft is not installed. Please use the 🧙🏽‍♂️ Install Wizard above.")


Syft is imported


In [2]:
try:
   domain_client = sy.login(
      port=8081,
      email="info@openmined.org",
      password="changethis"
   )
except Exception as e:
   print("Unable to login. Please check your domain is up with `!hagrid check localhost:8081`")

Logged into Xenodochial Isbell as <info@openmined.org>


In [7]:
import pandas as pd

In [24]:
dataset = sy.Dataset(name="John1",
                     description="tests",
                     asset_list=[
                         sy.Asset(name="ages",
                               data=pd.DataFrame(
                                   {
                                   'Patient_ID': ['014', '015', '022', '034', '044'],
                                   'Age': [60, 39, 35, 60, 25]
                                   }
                               ),
                               mock=pd.DataFrame(
                                   {
                                   'Patient_ID': ['1', '2', '3', '4', '5'],
                                   'Age': [60, 49, 45, 70, 35]
                                   }
                               ),
                               mock_is_real=False)
                     ]
                    )
domain_client.upload_dataset(dataset)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.02it/s]

Uploading: ages


<class 'syft.service.response.SyftSuccess'>: Dataset uploaded to 'Xenodochial Isbell'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [10]:
domain_client.register(name="Jane Doe", email="janedoe@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")

<class 'syft.service.response.SyftError'>: User already exists with email: janedoe@caltech.edu

In [25]:
ds_client = sy.login(
      port=8081,
      email="janedoe@caltech.edu",
      password="abc123"
   )

Logged into Xenodochial Isbell as <janedoe@caltech.edu>


In [27]:
asset = ds_client.datasets[-1].assets["ages"]

In [28]:
asset

syft.service.dataset.dataset.Asset

In [14]:
mock = asset.mock
mock

,Patient_ID,Age
0,1,50
1,2,49
2,3,45
3,4,70
4,5,35


In [29]:
age_sum = mock['Age'].mean()
print(age_sum)

49.8


In [33]:
@sy.syft_function_single_use(data=asset)
def get_mean_age1(df):
    return df['Age'].mean()

Syft function 'get_mean_age1' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.


In [34]:
req = ds_client.code.request_code_execution(get_mean_age1)
req

syft.service.request.request.Request

In [48]:
domain_client.requests

In [36]:
request = domain_client.requests[0]

In [37]:
str_changes = []
for change in request.changes:
    if change.id in request.current_change_state:
        print("A")
        str_change = (
            change.__repr_syft_nested__()
            if hasattr(change, "__repr_syft_nested__")
            else type(change)
        )
        str_change = f"{str_change}. "
        str_changes.append(str_change)
str_changes = "\n".join(str_changes)

In [38]:
usercode = request.code

In [39]:
usercode

```python
class UserCode
    id: str = 4193e82653e64a95822dbb8ff07ebcf5
    status.approved: str = False
    service_func_name: str = get_mean_age1
    code:

@sy.syft_function_single_use(data=asset)
def get_mean_age1(df):
    return df['Age'].mean()

```

In [41]:
get_mean_age_user_function = usercode.unsafe_function

In [42]:
asset = usercode.assets[0]
real_result = get_mean_age_user_function(df=asset)
print(real_result)

43.8


In [43]:
result = request.accept_by_depositing_result(real_result)
print(result)
assert isinstance(result, sy.SyftSuccess)

message='Request 7246728aacea4e0cb0954e3e11083928 changes applied'


In [44]:
asset = ds_client.datasets[0].assets[0]

In [45]:
ds_client.code[0].status

{NodeView(node_name='Xenodochial Isbell', verify_key=7ae8316e0c69f9f1e23cafd3d9d5257a0fe8cd786340629197a3fa6ece01724b): <UserCodeStatus.EXECUTE: 'execute'>}

In [46]:
result_ptr = ds_client.code.get_mean_age(df=asset)

In [47]:
real_result = result_ptr.get()
print(real_result)

39.8
